In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tensorboardX import SummaryWriter

from tqdm import tqdm

batch_size = 128
epochs = 50
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [2]:
from Tars.distributions import RealNVP, Normal
from Tars.models import ML
from Tars.utils import get_dict_values

In [3]:
kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

In [4]:
import torch
z_dim = 784

loc = torch.tensor(0.).to(device)
scale = torch.tensor(1.).to(device)
prior = Normal(loc=loc, scale=scale, var=["z"], dim=z_dim)

p = RealNVP(prior, var=["x"], in_features=z_dim, num_nn_layers=3, hidden_features=128, num_multiscale_layers=2, num_flow_layers=4, image=True)

In [5]:
p.to(device)

RealNVP(
  (dist): Normal()
  (flows): ModuleList(
    (0): MultiScaleLayer1D(
      (flows): ModuleList(
        (0): AffineCouplingLayer1D(
          in_features=784, pattern=0
          (layers): ModuleList(
            (0): Linear(in_features=784, out_features=128, bias=True)
            (1): Linear(in_features=128, out_features=128, bias=True)
            (2): Linear(in_features=128, out_features=1568, bias=True)
          )
          (batch_norms): ModuleList(
            (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (1): AffineCouplingLayer1D(
          in_features=784, pattern=1
          (layers): ModuleList(
            (0): Linear(in_features=784, out_features=128, bias=True)
            (1): Linear(in_features=128, out_features=128, bias=True)
            (2): Linear(in_features=128, out_features=1568, bias=T

In [6]:
model = ML(p, optimizer=optim.Adam, optimizer_params={"lr":1e-3})

In [7]:
def train(epoch):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(tqdm(train_loader)):
        data = data.to(device)
        lower_bound, loss = model.train({"x": data.view(-1, 784)})
        train_loss += loss
 
    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    return train_loss

In [8]:
def test(epoch):
    test_loss = 0
    for i, (data, _) in enumerate(test_loader):
        data = data.to(device)
        lower_bound, loss = model.test({"x": data.view(-1, 784)})
        test_loss += loss

    test_loss = test_loss * test_loader.batch_size / len(test_loader.dataset)
    print('Test loss: {:.4f}'.format(test_loss))
    return test_loss

In [9]:
def plot_reconstrunction(data):
    with torch.no_grad():
        z = p.sample_inv({"x": data.view(-1, 784)})
        recon_batch = p.sample(z, only_flow=True)["x"].view(-1, 1, 28, 28)
    
        comparison = torch.cat([data, recon_batch]).cpu()
        return comparison
    
def plot_image_from_latent(z_sample):
    with torch.no_grad():
        sample = p.sample({"z":z_sample}, batch_size=64, only_flow=True)["x"].view(-1, 1, 28, 28).cpu()
        return sample

In [10]:
writer = SummaryWriter()

z_sample = torch.randn(64, z_dim).to(device)
x_original, y_original = iter(test_loader).next()
x_original = x_original.to(device)
y_original = y_original.to(device)

for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss = test(epoch)
    
    recon = plot_reconstrunction(x_original[:8])
    sample = plot_image_from_latent(z_sample)

    writer.add_scalar('train_loss', train_loss.item(), epoch)
    writer.add_scalar('test_loss', test_loss.item(), epoch)      
    
    writer.add_image('Image_from_latent', sample, epoch)
    writer.add_image('Image_reconstrunction', recon, epoch)
    
writer.close()

100%|██████████| 469/469 [00:17<00:00, 27.40it/s]

Epoch: 1 Train loss: -2567.9695



  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: -3085.9038


100%|██████████| 469/469 [00:16<00:00, 28.80it/s]


...

Epoch: 49 Train loss: -3357.3765


  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: -3403.0464


100%|██████████| 469/469 [00:15<00:00, 30.08it/s]


Epoch: 50 Train loss: -3357.7859
Test loss: -3403.0200
